# Modelling Activity Transitions using Markov Model

In this notebook, we will create a model for activity transitions using first order Markov Models. A Markov model is a stochastic model describing a sequence of possible events in which the probability of each event depends only on the state attained in the previous event.

An event in our case, would be the acitivity that a person is eprforming at a venue and can be considered to be the category of the venue. To create a Markov model we need to build a transition graph. A node in this graph is the activity that is being performed. A directed link from one node to another represents the transition between those activities. The weight of the link is probability of this transition. 

This graph can be represented using a transistion matrix. A transition matrix is an N x N matrix, where each row represents an activity and each column represents the next activity to be performed.  An entry in this matrix represents the probability of the activity transition.

Using the data collected from the next venues endpoint of the Foursquare API, we can build such a transition matrix and use this to get to most probable activity to be performed next.

In [1]:

import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

print('Libraries imported.')

Libraries imported.


In [2]:
categories = pd.read_pickle("data/categories.pkl")
venues = pd.read_pickle("data/venues.pkl")
next_venues = pd.read_pickle('data/next_venues.pkl')

In [3]:
print("No. of Categories: {}".format(categories.shape[0]))
print("No. of Venues: {}".format(venues.shape[0]))
print("No, of Venue Transitions: {}".format(next_venues.shape[0]))

No. of Categories: 937
No. of Venues: 5549
No, of Venue Transitions: 19494


In [4]:
class ActivityTransitionModel:
    
    def __init__(self, categories, next_venues):
        self.categories = categories
        self.next_venues = next_venues
    
    '''
    Returns a stack of the heirarchy for the specified category
    '''
    def get_category_heirarchy(self, category_id, stack=None):
        if stack is None:
            stack = []

        stack.append(category_id)

        if pd.isnull(self.categories.loc[category_id, 'parent_id']):
            return stack

        return self.get_category_heirarchy(self.categories.loc[category_id, 'parent_id'], stack)

    '''
    Returns the category id at the specified level.
    Levels start from 1
    '''
    def get_category_at_level(self, category_id, level=1):
        levels = self.get_category_heirarchy(category_id)
        level_id = ''
        for i in range(0, level):
            if (len(levels) > 0):
                level_id = levels.pop()
            else:
                break
        return level_id

    '''
    Returns the category name
    '''
    def get_category_name(self, category_id):
        return self.categories.loc[category_id, 'name']

    '''
    Prints all the children of the category
    '''
    def print_category_children(self, category_id, level=0):
        tabs = "\t" * (level)
        print("{}-> {}".format(tabs, self.categories.loc[category_id, 'name']))
        children = self.categories[self.categories['parent_id'] == category_id]
        for index, child in children.iterrows():
            self.print_category_children(index, level + 1)

    '''
    Returns all the children of the category. The Level information would be lost
    '''
    def get_category_children(self, category_id, children=None):
        if  children is None:
            children = set()

        for index, child in self.categories[self.categories['parent_id'] == category_id].iterrows():
            children.add(index)
            self.get_category_children(index, children)
        return children
    
    '''
    Returns the number of times a transition from the activity_name to another activity ahs occured
    '''
    def get_next_activities_count(self, activity_name, level_agg=2):
        # get the current category
        category = self.categories[self.categories['name'] ==  activity_name]
        if category.empty:
            print("Activity not found")
            return None
        
        category_id = category.index[0]
        
        # identify all the current categories including the sub-categories
        category_ids = self.get_category_children(category_id)
        category_ids.add(category_id)

        # create a copy of next venues
        next_venues = self.next_venues[['venue_category', 'next_venue_category']].copy()
        
        # get the next categories
        next_venues = next_venues[next_venues['venue_category'].isin(category_ids)]
        
        # All the categories in the venue_category column are the category and all it's sub-categories.
        # We set the value in this column for all rows to the category_id of the activity
        # For the next_venues we aggregate upto the specified level
        next_venues['venue_category'] = next_venues['venue_category'].apply(lambda x: category_id)
        next_venues['next_venue_category'] = next_venues['next_venue_category'].apply(lambda x: self.get_category_at_level(x, level_agg))
        
        # get category names
        next_venues['venue_category'] = next_venues['venue_category'].apply(lambda x: self.get_category_name(x))
        next_venues['next_venue_category'] = next_venues['next_venue_category'].apply(lambda x: self.get_category_name(x))
        
        # pivot the table to get counts of transitions
        next_venues = pd.pivot_table(next_venues, index='venue_category', columns='next_venue_category', aggfunc=np.size)
        
        return next_venues
    
    '''
    Returns the probability of the activity transitions from the activity name to another activity
    '''
    def get_next_activities_probability(self, activity_name, level_agg=2):
        next_venues = self.get_next_activities_count(activity_name, level_agg)
        
        if next_venues is None or next_venues.empty:
            return None
        
        # calculate probabilities
        next_venues['total'] = next_venues.sum(axis=1)
        next_venues = next_venues.div(next_venues.loc[activity_name,"total"])
        
        next_venues.drop('total', axis=1, inplace=True)
        
        return next_venues
    
    '''
    Returns the top n activity transition with its probability for the specified activity_name
    Shows results in percentage if show_percent is set to True else shows probabilities
    '''
    def get_next_activities(self, activity_name, n=1, level_agg=2, show_percent=False):
        next_venues = self.get_next_activities_probability(activity_name, level_agg)
        
        if next_venues is None or next_venues.empty:
            return None
        
        next_venues.sort_values(activity_name, axis=1, inplace=True, ascending=False)
        
        if show_percent:
            next_venues = next_venues * 100
        
        columns = next_venues.columns[0:n]
        
        return next_venues[columns]
    
        
        

## Building the Markov Model

In [5]:
model = ActivityTransitionModel(categories, next_venues)

The first step to build the Markov model would be to get the number of times there is a transition from activity A to activity B. Let's take for example, that we are currently at a Brewery. We can then check the next_venues for Breweries and count the number of times there is a transition to the new activity from a Brewery. We show this below

In [6]:
model.get_next_activities_count("Brewery", level_agg=2)

next_venue_category,American Restaurant,Arcade,Asian Restaurant,Bar,Beach,Brewery,Burger Joint,Dessert Shop,Distillery,Eastern European Restaurant,Food & Drink Shop,Food Truck,Gastropub,Irish Pub,Island,Mexican Restaurant,Miscellaneous Shop,Movie Theater,Music Venue,Park,Performing Arts Venue,Pharmacy,Pizza Place,Plaza,Sandwich Place,Seafood Restaurant,Stadium,Winery
venue_category,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Brewery,14,2,2,78,4,52,2,8,2,2,12,2,6,4,2,10,2,2,2,10,2,2,6,2,2,4,8,2


Once we have our transitions in terms of counts, we can convert it probabilities. For this we first need to the total number of transitions. For the example of Brewery, it is sum of the row i.e. 246. The the probability of transition to a Bar is given by 78/246 which is 0.317073

In [7]:
model.get_next_activities_probability("Brewery", level_agg=2)

next_venue_category,American Restaurant,Arcade,Asian Restaurant,Bar,Beach,Brewery,Burger Joint,Dessert Shop,Distillery,Eastern European Restaurant,Food & Drink Shop,Food Truck,Gastropub,Irish Pub,Island,Mexican Restaurant,Miscellaneous Shop,Movie Theater,Music Venue,Park,Performing Arts Venue,Pharmacy,Pizza Place,Plaza,Sandwich Place,Seafood Restaurant,Stadium,Winery
venue_category,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Brewery,0.056911,0.00813,0.00813,0.317073,0.01626,0.211382,0.00813,0.03252,0.00813,0.00813,0.04878,0.00813,0.02439,0.01626,0.00813,0.04065,0.00813,0.00813,0.00813,0.04065,0.00813,0.00813,0.02439,0.00813,0.00813,0.01626,0.03252,0.00813


Finally to find the most likely next activities, we need to lookup the most probable activities. The table below shows the top 5 most probable activities after Brewery along with the probability (in percentage) of doing that activity

In [8]:
model.get_next_activities("Brewery", n=5, level_agg=2, show_percent=True)

next_venue_category,Bar,Brewery,American Restaurant,Food & Drink Shop,Mexican Restaurant
venue_category,,,,,
Brewery,31.707317,21.138211,5.691057,4.878049,4.065041


## Understanding the Level Aggregation

By specifying the level aggregation we can decide at what level we would like to see our results. For example when we specify different levels for Brewery to get the top 10 next activities, we see the below results

In [9]:
model.get_next_activities("Brewery", n=10, level_agg=1, show_percent=True)

next_venue_category,Nightlife Spot,Food,Outdoors & Recreation,Arts & Entertainment,Shop & Service,Professional & Other Places
venue_category,,,,,,
Brewery,52.845528,25.203252,7.317073,6.504065,6.504065,1.626016


In [10]:
model.get_next_activities("Brewery", n=10, level_agg=2, show_percent=True)

next_venue_category,Bar,Brewery,American Restaurant,Food & Drink Shop,Mexican Restaurant,Park,Dessert Shop,Stadium,Pizza Place,Gastropub
venue_category,,,,,,,,,,
Brewery,31.707317,21.138211,5.691057,4.878049,4.065041,4.065041,3.252033,3.252033,2.439024,2.439024


In [11]:
model.get_next_activities("Brewery", n=10, level_agg=3, show_percent=True)

next_venue_category,Brewery,Cocktail Bar,Bar,American Restaurant,Dive Bar,Beer Bar,Park,Baseball Stadium,Mexican Restaurant,Ice Cream Shop
venue_category,,,,,,,,,,
Brewery,21.138211,8.130081,7.317073,5.691057,4.878049,4.065041,4.065041,3.252033,3.252033,3.252033


As we specifiy a higher level, we drill down to the sub-categories and get estimates for it. While a lower level gives estimates for the main levels taking into consideration the sub-categories as well.

## The Transition Matrix

As an example we show the transition matrix for all the main categories

In [12]:
main_categories = categories[pd.isnull(categories["parent_id"])]
main_categories

,name,icon,parent_id
4d4b7104d754a06370d81259,Arts & Entertainment,https://ss3.4sqi.net/img/categories_v2/arts_en...,NaN
4d4b7105d754a06372d81259,College & University,https://ss3.4sqi.net/img/categories_v2/educati...,None
4d4b7105d754a06373d81259,Event,https://ss3.4sqi.net/img/categories_v2/event/d...,None
4d4b7105d754a06374d81259,Food,https://ss3.4sqi.net/img/categories_v2/food/de...,None
4d4b7105d754a06376d81259,Nightlife Spot,https://ss3.4sqi.net/img/categories_v2/nightli...,None
4d4b7105d754a06377d81259,Outdoors & Recreation,https://ss3.4sqi.net/img/categories_v2/parks_o...,None
4d4b7105d754a06375d81259,Professional & Other Places,https://ss3.4sqi.net/img/categories_v2/buildin...,None
4e67e38e036454776db1fb3a,Residence,https://ss3.4sqi.net/img/categories_v2/buildin...,None
4d4b7105d754a06378d81259,Shop & Service,https://ss3.4sqi.net/img/categories_v2/shops/d...,None
4d4b7105d754a06379d81259,Travel & Transport,https://ss3.4sqi.net/img/categories_v2/travel/...,None


In [45]:
transition_matrix = pd.DataFrame()
for index, row in main_categories.iterrows():
    result = model.get_next_activities(row['name'], n=20, level_agg=1)
    if result is not None:
        transition_matrix = transition_matrix.append(result)
#transition_matrix.fillna(0, inplace=True)
transition_matrix

,Arts & Entertainment,College & University,Food,Nightlife Spot,Outdoors & Recreation,Professional & Other Places,Shop & Service,Travel & Transport
venue_category,,,,,,,,
Arts & Entertainment,0.231465,NaN,0.303797,0.177215,0.147378,0.019892,0.096745,0.023508
College & University,0.055556,NaN,0.388889,0.055556,0.166667,0.055556,0.277778,NaN
Food,0.098659,0.000665,0.366589,0.181909,0.125707,0.009533,0.206518,0.010420
Nightlife Spot,0.072161,NaN,0.363792,0.514091,0.017506,0.005124,0.025192,0.002135
Outdoors & Recreation,0.098407,0.000937,0.259138,0.025305,0.379569,0.028585,0.189784,0.018276
Professional & Other Places,0.082569,NaN,0.362385,0.091743,0.270642,0.022936,0.151376,0.018349
Residence,NaN,NaN,NaN,NaN,1.000000,NaN,NaN,NaN
Shop & Service,0.023245,NaN,0.315567,0.029819,0.096032,0.006340,0.522423,0.006574
Travel & Transport,0.078481,NaN,0.379747,0.075949,0.222785,0.035443,0.139241,0.068354


## The Transition Graph

Using the transition matrix we can build a transition graph. I have used a force directed graph from d3.js to build the graph. We will first create the list a nodes and the links between the nodes

In [82]:
melted = pd.melt(transition_matrix.reset_index(), id_vars='venue_category', var_name='next_venue_category')
melted.dropna(inplace=True)
index = transition_matrix.index
links_list = list(melted.apply(lambda row: {"source": index.get_loc(row['venue_category']), "target": index.get_loc(row['next_venue_category']), "value": row['value']}, axis=1))

In [83]:
node_list =  [{"group": index.get_loc(cat), "name": cat} for cat in transition_matrix.index]

In [84]:
node_list

[{'group': 0, 'name': 'Arts & Entertainment'},
 {'group': 1, 'name': 'College & University'},
 {'group': 2, 'name': 'Food'},
 {'group': 3, 'name': 'Nightlife Spot'},
 {'group': 4, 'name': 'Outdoors & Recreation'},
 {'group': 5, 'name': 'Professional & Other Places'},
 {'group': 6, 'name': 'Residence'},
 {'group': 7, 'name': 'Shop & Service'},
 {'group': 8, 'name': 'Travel & Transport'}]

In [85]:
links_list

[{'source': 0, 'target': 0, 'value': 0.2314647377938517},
 {'source': 1, 'target': 0, 'value': 0.05555555555555555},
 {'source': 2, 'target': 0, 'value': 0.09865868528987917},
 {'source': 3, 'target': 0, 'value': 0.0721605465414176},
 {'source': 4, 'target': 0, 'value': 0.09840674789128398},
 {'source': 5, 'target': 0, 'value': 0.08256880733944955},
 {'source': 7, 'target': 0, 'value': 0.02324489316741019},
 {'source': 8, 'target': 0, 'value': 0.07848101265822785},
 {'source': 2, 'target': 1, 'value': 0.0006651147322913202},
 {'source': 4, 'target': 1, 'value': 0.0009372071227741331},
 {'source': 0, 'target': 2, 'value': 0.3037974683544304},
 {'source': 1, 'target': 2, 'value': 0.3888888888888889},
 {'source': 2, 'target': 2, 'value': 0.36658906994789936},
 {'source': 3, 'target': 2, 'value': 0.3637916310845431},
 {'source': 4, 'target': 2, 'value': 0.2591377694470478},
 {'source': 5, 'target': 2, 'value': 0.3623853211009174},
 {'source': 7, 'target': 2, 'value': 0.3155670345151444},
 

In [88]:
json_prep = {"nodes":node_list, "links":links_list}

dict_keys(['nodes', 'links'])

You can view the graph in [jsfiddle](http://jsfiddle.net/eserrao/fmd20bL4/)

In [98]:
%%html
<iframe width="100%" height="600" src="//jsfiddle.net/eserrao/fmd20bL4/embedded/" allowfullscreen="allowfullscreen" allowpaymentrequest frameborder="0"></iframe>